# Traitement et stockage des données issues du scraping

Vous avez scrapé les données du site de livres et les avez stockées dans un fichier. 

L'objectif de ce notebook est de créer une base de données pour y stocker ces données.

In [1]:
import sqlite3
import pandas as pd

Lire les données du fichier sauvegardé en utilisant pandas.

In [17]:
# Lire les données du fichier que vous venez d'enregistrer
df_books = pd.read_csv('books_infos.csv')
print(df_books)

FileNotFoundError: [Errno 2] No such file or directory: 'books_infos.csv'

## 1. Prétraitement des données

On souhaite créer la table _book_ contenant les attributs suivants : 
- id : INT, PK,
- title : TEXT,
- price : DECIMAL
- availability : BOOLEAN
- rating : INT [0:5]

Vérifier les types des colonnes du dataframe.

In [3]:
# Vérification des types de données
df_books.dtypes

title           object
price           object
rating          object
availability    object
dtype: object

Dans les cellules qui suivent, des méthodes de traitement de données sont suggérées pour donner un aperçu de ce qu'il est possible de faire avec pandas.

**Il est tout à fait possible de faire autrement.**

Utiliser la méthode pandas [_astype_](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.astype.html) pour convertir la colonne de titre en chaîne de caractère.

In [4]:
# Conversion de title en chaîne de caractères
df_books["title"] = df_books["title"].astype(str)

# Vérification du type de la colonne title
print(df_books["title"].dtype)

object


Pour convertir la colonne de prix en nombre décimal, il est nécessaire d'utiliser une étape intermédiaire pour retirer le caractère "£".

Il est possible par exemple d'utiliser l'attribut [.str](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.html) de la série "price".

In [5]:
# Convertir la colonne price en type décimal
df_books["price"] = df_books["price"].str.replace('£', "")
df_books["price"] = df_books["price"].str.replace('Â', "")
df_books["price"] = df_books["price"].str.strip()

# Vérification du type de la colonne price
print(df_books["price"].dtype)
print(df_books['price'])

object
0      51.77
1      53.74
2      50.10
3      47.82
4      54.23
       ...  
995    55.53
996    57.06
997    16.97
998    53.98
999    26.08
Name: price, Length: 1000, dtype: object


Convertir la colonne `availability` en boolen (True/False).

Quelles sont les valeurs possibles pour la colonne availability ?

In [6]:
# Valeurs possibles de la colonne availability
#  'In stock'


print(df_books['availability'])


0      In stock
1      In stock
2      In stock
3      In stock
4      In stock
         ...   
995    In stock
996    In stock
997    In stock
998    In stock
999    In stock
Name: availability, Length: 1000, dtype: object


Créer une fonction qui prend en entrée la valeur de `availability` et qui renvoie True ou False en fonction de la valeur d'entrée.

In [7]:
# Fonction pour convertir la valeur de availability en booléen
def convert_availability(value : str) -> bool:
    """Convert the availability value to a boolean.

    Args:
        value (str): The availability status of the book.

    Returns:
        bool: True if the book is available, False otherwise.
    """
    if value == 'In stock':
        return True
    else:
        return False

 
    

Utiliser la méthode [`apply`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.apply.html) pour appliquer la fonction à la colonne `availability`.

In [8]:
# Convertir la colonne availability en booléen (True/False)
df_books["availability"] = df_books['availability'].apply(convert_availability)

# Vérification du type de la colonne availability
print(df_books["availability"].dtype)

bool


Convertir la colonne _rating_ en chiffre en utilisant un dictionnaire `rating_map` et la méthode [_map_](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.map.html).

In [9]:
# Dictionnaire associant les notes au format initial et les valeurs numérique
ratings_map = {
    'Zero': 0,
    'One': 1,
    'Two': 2,
    'Three': 3,
    'Four': 4,
    'Five': 5
}

df_books["rating"] = df_books["rating"].map(ratings_map)

# Vérification du type de la colonne rating
print(df_books["rating"].dtype)

int64


In [10]:
# Créer une fonction convert_types qui combine les traitements faits dans les cellules précédentes
def convert_types(df_books: pd.DataFrame) -> pd.DataFrame:
    """Convert the types of the DataFrame columns to appropriate types.

    Args:
        df_books (pd.DataFrame): The DataFrame containing book data.

    Returns:
        pd.DataFrame: The DataFrame with converted types.
    """
    
    # Convertir en str
    df_books["title"] = df_books["title"].astype(str)

    # Nettoyer les prix (enlever le symbole £ et les caractères parasites) puis convertir en float
    df_books["price"] = df_books["price"].str.replace('£', "", regex=False)
    df_books["price"] = df_books["price"].str.replace('Â', "", regex=False)
    df_books["price"] = df_books["price"].str.strip()
    df_books["price"] = pd.to_numeric(df_books["price"], errors="coerce")
    df_books["price"] = df_books["price"].astype(float)
    
    # Convertir la disponibilité en booléen
    df_books["availability"] = df_books["availability"].apply(convert_availability)

    # Mapper les notes en chiffres
    ratings_map = {
        'Zero': 0,
        'One': 1,
        'Two': 2,
        'Three': 3,
        'Four': 4,
        'Five': 5
    }
    df_books["rating"] = df_books["rating"].map(ratings_map)

    

    return df_books

print(df_books.dtypes) # verif


title           object
price           object
rating           int64
availability      bool
dtype: object


In [ ]:
df_books = pd.read_csv('books_infos.csv')
convert_types(df_books)
df_books.to_csv("book_infos_cleaned.csv", index=False)

---
## 2. Insertion des données en base

Dans cette section :
- on créé une BDD sqlite  `book_store.db` (ou on se connecte à la base si elle existe déjà) en utilisant la bibliothèque python sqlite3,
- on insère les données prétraitées dans la BDD

Utiliser le [tutoriel](https://www.ionos.fr/digitalguide/sites-internet/developpement-web/sqlite3-avec-python/) pour l'utilisation de sqlite3.

Utiliser la fonction pandas adaptée qui permet d'insérer un dataframe dans une BDD.

In [ ]:
# Création de la BDD et insertion des données
connection = sqlite3.connect('book_store.db') # connection à la bdd
cursor = connection.cursor() # fonction permettant d'envoyer des requêtes SQL depuis python
df_books = pd.read_csv('book_infos_cleaned.csv')
if 'Unnamed: 0' in df_books.columns: # Ayant exporter les données en csv sans utiliser index=False, cette ligne permet supprimer l'ajout d'un nouvel index
    df_books = df_books.drop(columns=['Unnamed: 0'])
df_books.to_sql(
    name='book',
    con=connection,
    if_exists='replace',
    index=True,
    index_label='book_id'
)

1000

Vérifier le nombre de livres présents dans la BDD en utilisant sqlite3 et la requête SQL adaptée.

In [40]:
# Compter le nombre de livre dans la BDD
cursor.execute("SELECT COUNT(book_id) from book")
resultat = cursor.fetchone() # retourne le 1er résultat
print(f'Nombre total de livres :  {resultat[0]}') # Affichage du résultat

connection.commit()
connection.close()

Nombre total de livres :  1000
